In [6]:

import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree
import os, warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

BASE = '/content/drive/MyDrive/loan-data'
TEST_PRED_PATH = os.path.join(BASE, 'test_predictions.csv')
RL_PATH        = os.path.join(BASE, 'policy_results.csv')
TEST_PARQUET   = os.path.join(BASE, 'test.parquet')
OUT_PATH       = os.path.join(BASE, 'merged_rl_dl_by_test.csv')

print("Files in folder:")
!ls -lh /content/drive/MyDrive/loan-data || true
print("-"*60)

# load
test_pred = pd.read_csv(TEST_PRED_PATH)
rl = pd.read_csv(RL_PATH)
test_df = pd.read_parquet(TEST_PARQUET)

print("Loaded shapes -> test_pred:", test_pred.shape, ", test_df:", test_df.shape, ", rl:", rl.shape)
print("-"*60)
print("Columns in test_df (sample):", test_df.columns.tolist()[:40])
print("Columns in rl (sample):", rl.columns.tolist()[:40])
print("-"*60)

# Ensure test_pred aligns to test_df by order (this is expected)
if len(test_pred) != len(test_df):
    raise RuntimeError(f"Length mismatch: test_predictions ({len(test_pred)}) vs test.parquet ({len(test_df)}). They must match by row order. Check which file is the actual test split.")
# Attach predictions to test_df by order
test_df = test_df.reset_index(drop=True)
test_pred = test_pred.reset_index(drop=True)
test_df = pd.concat([test_df, test_pred], axis=1)
print("Attached predictions to test dataframe. New shape:", test_df.shape)

# Choose numeric features present in both to match on
candidate_features = ['loan_amnt','loan_amount','loan_amnt_inv','funded_amnt','funded_amnt_inv','int_rate','interest_rate','annual_inc','dti','fico_range_low','fico_range_high','term']
# select features that appear in both test_df and rl
features = []
for f in ['loan_amnt','int_rate','annual_inc','dti','fico_range_low','fico_range_high','term']:
    if f in test_df.columns and f in rl.columns:
        features.append(f)
# If none, try alternative names
if len(features)==0:
    for f in ['loan_amount','interest_rate','funded_amnt']:
        if f in test_df.columns and f in rl.columns:
            features.append(f)

if len(features)==0:
    raise RuntimeError("No suitable numeric feature intersection found between test.parquet and policy_results.csv. Inspect column names.")

print("Using features for matching:", features)
# prepare matrices
A = test_df[features].fillna(0).astype(float).values  # test rows (n_test x d)
B = rl[features].fillna(0).astype(float).values       # rl rows (n_rl x d)
print("Matrix shapes A (test):", A.shape, " B (rl):", B.shape)

# Normalize columns by pooled std to avoid scale domination
stack = np.vstack([A, B])
std = stack.std(axis=0, ddof=0)
std[std == 0] = 1.0
A_s = A / std
B_s = B / std

# If target exists, we will match within same target class to avoid mismatching paid vs charged-off rows
use_target = ('target' in test_df.columns) and ('target' in rl.columns)
if use_target:
    print("Will match only within same target (paid/default).")
else:
    print("No 'target' column found in one of the files. Matching across all rows (less strict).")

# Build KDTree per target value if available, else single tree
mapped_indices = np.full(len(test_df), -1, dtype=int)
mapped_dists = np.full(len(test_df), np.nan, dtype=float)

if use_target:
    unique_targets = np.unique(test_df['target'].astype(int).values)
    for t in unique_targets:
        idx_A = np.where(test_df['target'].astype(int).values == int(t))[0]
        idx_B = np.where(rl['target'].astype(int).values == int(t))[0]
        if len(idx_B) == 0:
            # fallback: allow matching across all rl rows if none with same target
            idx_B = np.arange(B_s.shape[0])
        A_block = A_s[idx_A]
        B_block = B_s[idx_B]
        tree = KDTree(B_block)
        dists, idxs = tree.query(A_block, k=1)
        mapped_indices[idx_A] = idx_B[idxs[:,0]]
        mapped_dists[idx_A] = dists[:,0]
else:
    tree = KDTree(B_s)
    dists, idxs = tree.query(A_s, k=1)
    mapped_indices = idxs[:,0]
    mapped_dists = dists[:,0]

# Build matched RL rows for each test row
matched_rl = rl.reset_index(drop=True).iloc[mapped_indices].reset_index(drop=True)
# combine
merged = pd.concat([test_df.reset_index(drop=True), matched_rl.add_suffix('_rl')], axis=1)
merged['match_dist'] = mapped_dists

# Diagnostics
print("Matched rows:", merged.shape[0])
print("Match distance stats (min, 25%, median, 75%, max):",
      np.nanmin(mapped_dists), np.nanpercentile(mapped_dists,25), np.nanmedian(mapped_dists), np.nanpercentile(mapped_dists,75), np.nanmax(mapped_dists))
# Show some samples where distance is high (possible bad matches)
threshold_high = np.nanpercentile(mapped_dists, 95)
bad_matches = merged[merged['match_dist'] > threshold_high]
print("Rows with match_dist > 95th percentile:", len(bad_matches))
if len(bad_matches) > 0:
    display(bad_matches.head(6))

# Quick check: how many merged rows have identical loan_amnt and int_rate (exact match)
exact_match_mask = np.ones(len(merged), dtype=bool)
for f in features:
    exact_match_mask &= np.isclose(merged[f].astype(float).values, merged[f + '_rl'].astype(float).values)
exact_matches = exact_match_mask.sum()
print("Exact matches on all chosen features:", int(exact_matches), " / ", len(merged))

# Save result
merged.to_csv(OUT_PATH, index=False)
print("Saved merged file to:", OUT_PATH)
print("Open it in Drive and inspect first 50 rows. If matches look correct, proceed with comparison analysis.")


Files in folder:
total 1.8G
-rw------- 1 root root 1.6G Dec 12 09:01 accepted_2007_to_2018.csv
-rw------- 1 root root  80M Dec 12 09:39 accepted_sample.csv
-rw------- 1 root root  80M Dec 12 09:11 accepted_sample_stratified_5pct.csv
-rw------- 1 root root 459K Dec 12 10:54 best_mlp_checkpoint.pt
drwx------ 2 root root 4.0K Dec 12 11:02 models
-rw------- 1 root root  21M Dec 12 11:02 policy_results.csv
-rw------- 1 root root 2.5M Dec 12 11:02 policy_results.parquet
-rw------- 1 root root 1.1M Dec 12 10:28 processed_sample.parquet
-rw------- 1 root root 249K Dec 12 10:29 test.parquet
-rw------- 1 root root 366K Dec 12 10:54 test_predictions.csv
-rw------- 1 root root 853K Dec 12 10:29 train.parquet
------------------------------------------------------------
Loaded shapes -> test_pred: (22607, 3) , test_df: (22607, 15) , rl: (113034, 19)
------------------------------------------------------------
Columns in test_df (sample): ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'fico_range_low

,loan_amnt,int_rate,annual_inc,dti,fico_range_low,fico_range_high,term_ 36 months,term_ 60 months,home_ownership_ANY,home_ownership_MORTGAGE,...,home_ownership_NONE_rl,home_ownership_OTHER_rl,home_ownership_OWN_rl,home_ownership_RENT_rl,target_rl,policy_action_rl,q0_rl,q1_rl,policy_reward_rl,match_dist
4,-0.983666,-1.062133,-0.649249,0.197137,0.797170,0.797136,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0,1,0.0,0.026049,0.010448,2.482534e-16
24,2.166504,-0.640744,2.138744,-1.310840,3.061577,3.061468,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0,0,0.0,-0.054399,0.000000,4.440892e-16
43,0.102600,1.435060,-0.079061,-0.201661,-1.165317,-1.165286,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0,0,0.0,-0.020834,0.000000,2.562698e-16
84,-0.983666,-1.060057,-0.356287,-0.901492,1.099091,1.099047,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0,1,0.0,0.020788,0.010427,2.543841e-16
94,-1.048842,-0.972873,-0.415693,0.023410,-0.259554,-0.259553,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,1,0.0,0.050569,0.010204,2.460129e-16
152,1.080238,0.702301,1.082646,-1.319984,2.004854,2.004780,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0.0,-0.113613,0.000000,4.965068e-16


Exact matches on all chosen features: 22607  /  22607
Saved merged file to: /content/drive/MyDrive/loan-data/merged_rl_dl_by_test.csv
Open it in Drive and inspect first 50 rows. If matches look correct, proceed with comparison analysis.
